In [ ]:
# Spectral plot
import sys
sys.path.insert(0,"..")

import speasy as spz
import matplotlib.pyplot as plt
import numpy as np
import speasy_utils

In [ ]:
param_id = "thb_ion_sp"
start = "2008/05/16T05:00:00"
stop = "2008/05/16T15:00:00"

In [ ]:
# get data, see what shape it has
data = spz.get_data(f"amda/{param_id}", start, stop)
print(f"Data shape : {data.data.shape}")

In [ ]:
import matplotlib
import matplotlib.colors as colors
import matplotlib.ticker as mticker
from datetime import datetime

def spectro_plot(param_id, start, stop, ylabel=None, channels = None, ax=None, figsize=(10,2), 
                 vmin=None, vmax=None, lognorm=True):
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=figsize)
    # get the data
    param_data = spz.get_data(param_id, start, stop)
    [n,m] = param_data.data.shape
    X = param_data.data   
    
    # channels (constant channels case)
    if channels is None:
        y = np.arange(0,m,1)
    else:
        y = channels
    
    # grid
    x1, y1 = np.meshgrid(param_data.time,y, indexing="ij")
    
    # data bounds
    if vmin is None:
        vmin = np.nanmin(X)
    if vmax is None:
        vmax = np.nanmax(X)
    
    # colormap
    cmap = matplotlib.cm.rainbow.copy()
    cmap.set_bad('White',0.)
    
    # normalize colormapping
    if lognorm and vmin>0.:
        norm=colors.LogNorm(vmin=vmin, vmax=vmax)
    else:
        norm=None
    
    
    c = ax.pcolormesh(x1, y1, X, cmap=cmap, norm=norm, edgecolors="face")
    plt.colorbar(c,ax=ax, norm=norm)
    
    ax.set_xlabel("Time")
    x_ticks = ax.get_xticks()
    x_ticks = [datetime.utcfromtimestamp(xi) for xi in x_ticks]
    x_labels = [d.strftime("%H:%M") for d in x_ticks]
    
    ticks_loc = ax.get_xticks().tolist()
    ax.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
    ax.set_xticklabels(x_labels)
    
    if ylabel:
        ax.set_ylabel(ylabel)
    
    ax.set_ylim(y.min(), y.max())
    
    return ax
    


In [ ]:
ybin_min = np.array([22680,18275,13883,10546,8012,6086,4624,3512,
                    2668,2027,1540,1170,888,675,512,390,296,225,
                    170,130,98.9,75.2,57.,43.3,33.,25.1,18.9,
                    14.2,11.,8.44,6.38,4.88])
ybin_max = np.hstack((np.array([26501]), ybin_min[:-1]))
ybins = np.vstack((ybin_min, ybin_max)).T
channels = np.mean(ybins, axis=1)

ax = spectro_plot("amda/thb_ion_sp", start, stop, channels=channels, ylabel="Energy", vmin=1e2, vmax=1e8)
ax.set_yscale("log")
plt.show()